In [1]:
#chack for updated files\n,
%load_ext autoreload
%autoreload 2
#import packages
from __future__ import division

import numpy as np
import glob, json
import pickle
import os as os_pack
import matplotlib.pyplot as plt
import corner
#%matplotlib inline\n",
%config InlineBackend.figure_format = 'retina'
import healpy as hp
import os, glob, json, pickle
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg as sl
import enterprise
from enterprise.pulsar import Pulsar
import enterprise.signals.parameter as parameter
from enterprise.signals import utils
from enterprise.signals import signal_base
from enterprise.signals import selections
from enterprise.signals.selections import Selection
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
from enterprise.signals import deterministic_signals
import enterprise.constants as const
from enterprise_extensions import blocks
from enterprise_extensions import models as ee_models
from enterprise_extensions import model_utils as ee_model_utils
from enterprise_extensions import model_orfs
from enterprise_extensions.frequentist import optimal_statistic as opt_stat
from enterprise_extensions import sampler as ee_sampler
from enterprise.signals.signal_base import LogLikelihood
import enterprise_wavelets as models
from enterprise.signals.deterministic_signals import Deterministic
from enterprise.signals.parameter import function
from la_forge.core import Core
from la_forge.diagnostics import plot_chains
from la_forge import rednoise
import la_forge
import corner
from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc
    #style

In [50]:
#Loading in pickle and noise files
pint_pickle = '/home/reyna/OS_15yr/15yr_data/v1p1_de440_pint_bipm2019.pkl'
noise_file = '/home/reyna/OS_15yr/15yr_data/v1p1_wn_dict.json'
#psrlist = np.loadtxt('/home/reyna/15yr_v1p0/15yr_v1-20211001T235643Z-001/15yr_v1/psrlist_15yr_pint.txt', dtype = str)
with open(noise_file, 'r') as h:
    noise_params = json.load(h)
with open(pint_pickle,'rb') as f:
    allpsrs = pickle.load(f)
psrs = []
for ii,p in enumerate(allpsrs):
    psrs.append(p)
#Temporary to get code to not crash
psrs = psrs[4:8]
psrlist = [psr.name for psr in psrs]
print(psrlist)
for i in range(len(psrs)):
    print((max(psrs[i].toas) - min(psrs[i].toas))/(3.17*10**(7)))

['J0030+0451', 'J0340+4130', 'J0406+3039', 'J0437-4715']
15.460752938661816
8.097196263848735
3.5471797714338695
4.73231500896138


In [51]:
glitches = []
log10_f0 = parameter.Uniform(np.log10(3.5e-9), np.log10(1e-7))("Glitch_"+str(1)+'_'+'log10_f0')
phase0 = parameter.Uniform(0, 2*np.pi)("Glitch_"+str(1)+'_'+'phase0')
tau = parameter.Uniform(0.2, 5)("Glitch_"+str(1)+'_'+'tau')
t0 = parameter.Uniform(0.0, 10.0)("Glitch_"+str(1)+'_'+'t0')
psr_idx = parameter.Uniform(-0.5, len(psrs)-0.5)("Glitch_"+str(1)+'_'+'psr_idx')
log10_h = parameter.LinearExp(-6.5, -5)("Glitch_"+str(1)+'_'+'log10_h')
glitch_wf = models.glitch_delay(log10_h = log10_h, tau = tau, log10_f0 = log10_f0, t0 = t0, phase0 = phase0, tref=53000*86400,
                                        psr_float_idx = psr_idx, pulsars=psrs)
glitches.append(deterministic_signals.Deterministic(glitch_wf, name='Glitch'+str(1) ))

In [52]:
tm = gp_signals.TimingModel(use_svd=True)
wn = blocks.white_noise_block(vary=False, inc_ecorr=False)
#s = base_model
s = tm + wn + glitches[0]
model = []
for p in psrs:
    model.append(s(p))
with open(noise_file, 'r') as fp:
    noisedict = json.load(fp)
    pta = signal_base.PTA(model)
    pta.set_default_params(noisedict)

In [53]:
d0 = parameter.sample(pta.params)
x0 = np.array([d0[par.name] for par in pta.params])

In [54]:
pta.get_lnlikelihood(x0)

449134.2142376422

In [31]:
449120.3574538504
449119.80307398614

282505.0135900155

In [58]:
d0

{'Glitch_1_log10_f0': -8.150958196881906,
 'Glitch_1_log10_h': -5.024028621193761,
 'Glitch_1_phase0': 1.9205752139963215,
 'Glitch_1_psr_idx': 3.4251082243738007,
 'Glitch_1_t0': 1.2540172065765909,
 'Glitch_1_tau': 4.866613926953403}

In [55]:
import Fast_Burst_likelihood as FB

In [56]:
FB1 = FB.FastBurst(pta = pta, psrs = psrs, params = d0, Npsr = len(psrs), tref=53000*86400)
#FB2 = FB.FastBurst(pta = pta, psrs = psrs, params = d0, Npsr = len(psrs), tref=53000*86400)

Nvecs arary:  [[4.61695300e-12 5.66353407e-11 6.83697178e-12 ... 9.39287544e-12
 1.01635764e-11 6.60318228e-12], [3.58685902e-11 2.77047195e-11 8.77407399e-11 ... 4.00248661e-11
 7.61997117e-11 4.82576810e-11], [2.30848299e-11 2.25095542e-12 6.90732193e-12 ... 1.26507575e-11
 3.65281122e-11 1.93290456e-11], [5.49729188e-12 4.98164427e-12 5.77752577e-12 ... 4.40088221e-12
 4.86261323e-12 1.46668814e-09], ...]
475710.6309232539
logdet_phi:  27907.331327087828
logdet_sigma:  8021.617645456276
dotSigmaTNr:  6531.363363038373
logdet_phi:  10960.30504265166
logdet_sigma:  2874.207139480091
dotSigmaTNr:  3.141037602664169
logdet_phi:  4973.583800867136
logdet_sigma:  1431.8168187786955
dotSigmaTNr:  0.018103655871505017
logdet_phi:  4513.066782268329
logdet_sigma:  1402.6832939290277
dotSigmaTNr:  2369.5424074152133
449120.3574538504


In [57]:
FB1.get_lnlikelihood(10**(x0[1]), x0[2], 10**(x0[0]), (3.15*10**7)*x0[5], (3.15*10**7)*x0[4], x0[3])

Amplitude:  9.46174803804544e-06
Frequency:  7.063855442983834e-09
glitch_index:  3.4251082243738007
Old Sigma:  [0. 0.]
New sigma:  [-3.24244730e-06 -8.88882509e-06]
Old M and N:  [[0. 0.]
 [0. 0.]] [0. 0.]
Input time:  457.1937732310488
Cosine:  [0. 0. 0. ... 0. 0. 0.]
Sine:  [0. 0. 0. ... 0. 0. 0.]
MM matrix: [[[0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]]]
NN matrix: [[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
Cosine:  [0. 0. 0. ... 0. 0. 0.]
Sine:  [0. 0. 0. ... 0. 0. 0.]
MM matrix: [[[0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]]]
NN matrix: [[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
Cosine:  [0. 0. 0. ... 0. 0. 0.]
Sine:  [0. 0. 0. ... 0. 0. 0.]
MM matrix: [[[0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]]]
NN matrix: [[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
Cosine:  [-3.26233207e-03 -3.26233207e-03 -3.26233207e-03 ... -4.17588904e-05
 -4.17588904e-05 -4.17588904e-05]
Sine:  [1.060184

449119.80307398614

In [12]:
482127.23089051904 #Bence version

482127.23089051904

In [13]:
448629.5379522237 #Enterprise version

448629.5379522237

In [14]:
LogL:  447940.2057596576
LogL:  447934.2907498001
LogL:  447934.2692285934
LogL:  447934.6153235128
447934.6153235128

447934.6153235128